# Introduction 

This jupyter notebook contains a demo version of our SiamFC tracker. 
It is designed to work in Google Colab \

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/zak-souid/DLAV_project/blob/main/Comb.ipynb)


It works in conjuction with our milestone 1 but everything is embedded in this notebook.

#### Initial imports

In [2]:
from google.colab import drive
import os
drive.mount('/content/drive')
!pip install torch torchvision got10k opencv-python
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from base64 import b64decode, b64encode
import cv2
import numpy as np
import PIL
import io
import html
import time
import torch
import pandas as pd
import os
import sys
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from collections import namedtuple
from torch.optim.lr_scheduler import ExponentialLR
from torch.utils.data import DataLoader
from got10k.trackers import Tracker


# load the files for milestone 2
if not os.path.exists('DLAV_project/siamfc-pytorch/'):
  !git clone --branch milestone2 https://github.com/zak-souid/DLAV_project.git ;

import sys
# load the files for milestone 1

obj_model = torch.hub.load('ultralytics/yolov5', 'yolov5s', pretrained=True)
hand_model = torch.hub.load('ultralytics/yolov5', 'custom', path='DLAV_project/best.pt')

sys.path.append('DLAV_project/siamfc-pytorch/')

from siamfc import ops
from siamfc.backbones import AlexNetV1
from siamfc.heads import SiamFC
from siamfc.losses import BalancedLoss
from siamfc.datasets import Pair
from siamfc.transforms import SiamFCTransforms
from siamfc.siamfc import Net
from siamfc.siamfc import TrackerSiamFC
import glob
from google.colab.patches import cv2_imshow

Mounted at /content/drive
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 87 kB 8.6 MB/s 
  Created wheel for got10k: filename=got10k-0.1.3-py3-none-any.whl size=43871 sha256=0b131819223b5c761046f57130216d2c69f77ecffbe0aae658365d8dd247a197
  Stored in directory: /root/.cache/pip/wheels/62/fe/c9/2a3cfd209474f1da9b4a1b2261488fbf55e975be561295874d
  Created wheel for fire: filename=fire-0.4.0-py2.py3-none-any.whl size=115942 sha256=409dd51ff30ef2fac16ca0c2cd39750f18996e6ce303b543770454d65edc6257
  Stored in directory: /root/.cache/pip/wheels/8a/67/fb/2e8a12fa16661b9d5af1f654bd199366799740a85c64981226
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=1b3df7463e94aae3f3815995a88e758ddeface5a1332e2bc597effc3a8e0f3ca
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built got10k fire wget
Cloning into 'DLAV

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to /root/.cache/torch/hub/master.zip
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: PyYAML
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-6-7 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)



  0%|          | 0.00/14.1M [00:00<?, ?B/s]


Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients
Adding AutoShape... 
Using cache found in /root/.cache/torch/hub/ultralytics_yolov5_master
requirements: PyYAML>=5.3.1 not found and is required by YOLOv5, attempting auto-update...
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

requirements: 1 package updated per /root/.cache/torch/hub/ultralytics_yolov5_master/requirements.txt
requirements: ⚠️ Restart runtime or rerun command for updates to take effect

YOLOv5 🚀 2022-6-7 Python-3.7.13 torch-1.11.0+cu113 CUDA:0 (Tesla T4, 15110MiB)

Fusing layers... 
Model summary: 213 layers, 7012822 parameters, 0 gradients
Adding AutoShape... 


#### Helper functions for the camera in colab

In [3]:
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
        js_reply: JavaScript object containing image from webcam
  Returns:
        img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
        bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [4]:
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[1].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment" }});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

#### Functions for Milestone 1

In [5]:
# Some functions used in next cell

def obj_detect(img):
    """
    Params:
          img: OpenCV BGR image
    Returns:
          res: pandas dataframe containing all objects detected by the model
    """
    results = obj_model(img)
    res = results.pandas().xyxy[0]  # img1 predictions (pandas)
    return res

def get_persons(objects):
    """
    Params:
          objects: pandas dataframe containing different objects
    Returns:
          persons: pandas dataframe containing only the objects labeled as person
    """
    persons = objects.loc[objects['name'] == "person"]
    idx_del = []
    areas = []

    for idx, pers in persons.iterrows():
        dx = pers["xmax"]-pers["xmin"]
        dy = pers["ymax"]-pers["ymin"]
        areas.append(dx*dy)
        if(areas[-1] < 10000):
            idx_del.append(idx)
    persons = persons.drop(idx_del)
    persons = persons.reset_index()


    return persons, areas

def get_personOfInterest(persons, hand, persons_area):  # returns None if rectangles don't intersect
    """
    Params:
          persons: pandas dataframe containing only objects labeled as person
          hand: pandas dataframe containing only the detected hands (labeled as 5)
    Returns:
          idx_max: integer corresponding to the index in the persons dataframe that corresponds to the person of interest
                   is equal to None if no hand is being detected or the hand does not belongs to any of the detectet persons
    """
    margin = 50 # For numerical inaccuracies
    idx_max = 0
    area_max = 0
    a_min = [-1,-1]
    a_max = [-1,-1]
    if not hand.empty:
        for idx, pers in persons.iterrows():
            dx = np.minimum(pers["xmax"], hand["xmax"].iloc[0]) - np.maximum(pers["xmin"], hand["xmin"].iloc[0])
            dy = np.minimum(pers["ymax"], hand["ymax"].iloc[0]) - np.maximum(pers["ymin"], hand["ymin"].iloc[0])
            if(dx<0 and dy<0):
                area = -dx*dy
            else:
                area = dx*dy
            if (area >= area_max - margin and persons_area[idx] >= persons_area[idx_max]):
                area_max = area
                idx_max = idx
                # actual overlapping area
                a_min = [int(np.maximum(pers["xmin"], hand["xmin"].iloc[0])),int(np.maximum(pers["ymin"], hand["ymin"].iloc[0]))]
                a_max = [int(np.minimum(pers["xmax"], hand["xmax"].iloc[0])),int(np.minimum(pers["ymax"], hand["ymax"].iloc[0]))]

    if (area_max>0):
        return idx_max, a_min, a_max
    else:
        return None, a_min, a_max

def hand_detect(img):
    """
    Params:
          img: OpenCV BGR image
    Returns:
          res: pandas dataframe containing all objects detected by the model
    """
    results = hand_model(img)
    res = results.pandas().xyxy[0]  # img1 predictions (pandas)
    return res

init = False
persOfInterest = False
bb_persOfInterest = [0,0,0,0]

def detect(img):
    global persOfInterest
    global bb_persOfInterest
    # detect all objects
    obj = obj_detect(img)
    # keep only the persons
    pers, persons_area = get_persons(obj)
    # detect hand
    hand = hand_detect(img)

    # identify person of interest
    idx_max, area_min, area_max = get_personOfInterest(pers,hand, persons_area)
    
    # create transparent overlay for bounding box
    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    
    # draw bounding box of the hand (if multiple hands, just the first one)
    if (not hand.empty) and area_min!=None:
        bbox_array = cv2.rectangle(bbox_array,(int(hand["xmin"].iloc[0]),int(hand["ymin"].iloc[0])), (int(hand["xmax"].iloc[0]),int(hand["ymax"].iloc[0])), (0, 0, 255), 2)
        bbox_array = cv2.rectangle(bbox_array,tuple(area_min), tuple(area_max), (255, 0, 255), 2)

    # draw bounding boxes on overlay
    for index, row in pers.iterrows():
        start_point = (int(row["xmin"]), int(row["ymin"]))
        end_point = (int(row["xmax"]), int(row["ymax"]))
        if (index == idx_max):
            color = (0, 255, 0)
            persOfInterest = True
            bb_persOfInterest = [start_point[0], start_point[1], end_point[0]-start_point[0], end_point[1]-start_point[1]]
            #print(bb_persOfInterest)
        else:
            color = (255, 0, 0)
        thickness = 2
        bbox_array = cv2.rectangle(bbox_array,start_point, end_point, color, thickness)

    return bbox_array

# SIAMFC

In [6]:

net_path = 'DLAV_project/siamfc_alexnet_e50.pth'
tracker = TrackerSiamFC(net_path=net_path)

def tracking(img):
    global init
    global tracker
    boxes = np.zeros([2,4])
    if not init:
        tracker.init(img, bb_persOfInterest)
        init = True
    boxes[0,:]=bb_persOfInterest
    # convert JS response to OpenCV Image
    #pair_img = [img_0, last_img, img]
    #boxes, times = tracker.track(pair_img, bb_persOfInterest, visualize=False)
    #print(boxes[1])
    
    boxes[-1, :] = tracker.update(img)
    start_point = (int(boxes[-1][0]),int(boxes[-1][1]))
    #print(start_point)
    end_point = (int(boxes[-1][0])+int(boxes[-1][2]),int(boxes[-1][1])+int(boxes[-1][3]))
    #print(end_point)

    bbox_array = np.zeros([480,640,4], dtype=np.uint8)
    bbox_array = cv2.rectangle(bbox_array, start_point, end_point, (255, 255, 0), 2) 
    last_img=img
    return bbox_array

### Video feed

In [7]:
def start_demo():

# start streaming video from webcam
  video_stream()
  # label for video
  label_html = 'Capturing...'
  # initialze bounding box to empty
  bbox = ''
  count = 0
  init = False

  while True:
    js_reply = video_frame(label_html, bbox)
    if not js_reply:
        break
    # convert JS response to OpenCV Image
    img = js_to_image(js_reply["img"])

    if not persOfInterest:
         bbox_array = detect(img)

    else:
         bbox_array = tracking(img)

    bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255
    # convert overlay of bbox into bytes  
    bbox_bytes = bbox_to_bytes(bbox_array)
    # update bbox so next frame gets new overlay
    bbox = bbox_bytes

# Demo

In [11]:
start_demo()

<IPython.core.display.Javascript object>

KeyboardInterrupt: ignored